### Install keras and tensorflow:

In [0]:
!conda install -c conda-forge keras --yes
!conda install -c conda-forge tensorflow --yes

### Import libraries:

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import os

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Data import:

In [0]:
folder_path = 'food/'

img_width, img_height = 224, 224

# load all images into a list
file_list = sorted(os.listdir(folder_path))
images = []
for img in file_list:
    if img == '.DS_Store':  # ignore stupid fookin mac file that wont go away
        continue
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)

### Load pre-trained model:

In [0]:
model = ResNet50(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


### Predict:

In [0]:
predictions = model.predict(images, batch_size=64, verbose=1)

10000/10000 [==============================] - 3732s 373ms/sample


### Save:

In [0]:
# Save top 3 label probabilities as the 3 features describing image:
predictions_decoded = np.array(decode_predictions(predictions, top=3))
predictions_decoded = predictions_decoded[:, :, 1]

df = pd.DataFrame(predictions_decoded, columns=['first', 'second', 'third'])
print(df)

df.to_csv('food_features', index=False)

           first         second         third
0        thimble      trilobite        packet
1         potpie         bakery         dough
2      ice_cream  mashed_potato         dough
3      ice_cream          dough     meat_loaf
4        burrito        pretzel     carbonara
...          ...            ...           ...
9995   ice_cream     neck_brace      lipstick
9996  strawberry      ice_cream      baseball
9997   meat_loaf          pizza  wooden_spoon
9998   soup_bowl     frying_pan         loupe
9999       dough           wool        mortar

[10000 rows x 3 columns]


In [0]:
'''img_path = 'food/00000.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
pred = np.array(decode_predictions(pred, top=3))
print(pred)'''

[[['n04423845' 'thimble' '0.7259098']
  ['n01768244' 'trilobite' '0.102758445']
  ['n03871628' 'packet' '0.019814285']]]


# Load food features and one hot encode:

Now we need to load the newly generated features, train a neural net on the triplets with the new features and see what happens:)

Load food features:

In [74]:
df = pd.read_csv("food_features") 
print(df)
print(df.iloc[2678, :])

           first         second         third
0        thimble      trilobite        packet
1         potpie         bakery         dough
2      ice_cream  mashed_potato         dough
3      ice_cream          dough     meat_loaf
4        burrito        pretzel     carbonara
...          ...            ...           ...
9995   ice_cream     neck_brace      lipstick
9996  strawberry      ice_cream      baseball
9997   meat_loaf          pizza  wooden_spoon
9998   soup_bowl     frying_pan         loupe
9999       dough           wool        mortar

[10000 rows x 3 columns]
first           tray
second    strawberry
third       cucumber
Name: 2678, dtype: object


Generate dataset:

In [96]:
df_triplets = pd.read_csv('train_triplets.txt', sep=" ", header=None)
print(df_triplets)

a = np.empty(shape=(df_triplets.shape[0],9), dtype='object')

for i in range(3):
  for idx, x in df_triplets[i].iteritems():
    a[idx, 3*i] = df.iloc[x, 0]
    a[idx, 3*i+1] = df.iloc[x, 1]
    a[idx, 3*i+2] = df.iloc[x, 2]

df_triplets['A_1'], df_triplets['A_2'], df_triplets['A_3'] = [a[:, 0], a[:, 1], a[:, 2]]
df_triplets['B_1'], df_triplets['B_2'], df_triplets['B_3'] = [a[:, 3], a[:, 4], a[:, 5]]
df_triplets['C_1'], df_triplets['C_2'], df_triplets['C_3'] = [a[:, 6], a[:, 7], a[:, 8]]

df_triplets = df_triplets.drop(columns=[0, 1, 2])

print(df_triplets)

          0     1     2
0      2461  3450  2678
1      2299  2499  4987
2      4663  1056  3029
3      4532  1186  1297
4      3454  3809  2204
...     ...   ...   ...
59510   466  2952  2530
59511  2646  3580  2359
59512  3255  4844  4334
59513  2136  4619   161
59514  2509  2552  3406

[59515 rows x 3 columns]
                 A_1            A_2  ...               C_2           C_3
0          ice_lolly     strawberry  ...        strawberry      cucumber
1      mashed_potato      ice_cream  ...               wok  wooden_spoon
2             orange            hip  ...        rotisserie         plate
3            pretzel          bagel  ...      coral_fungus     gyromitra
4              acorn            hip  ...  butternut_squash  acorn_squash
...              ...            ...  ...               ...           ...
59510          plate       broccoli  ...          broccoli         plate
59511   coral_fungus          dough  ...        frying_pan          corn
59512      soup_bowl        h

One-hot encode:

In [97]:
df_triplets_oh = pd.get_dummies(data=df_triplets)
print(df_triplets_oh)

       A_1_African_chameleon  A_1_African_crocodile  ...  C_3_wreck  C_3_zucchini
0                          0                      0  ...          0             0
1                          0                      0  ...          0             0
2                          0                      0  ...          0             0
3                          0                      0  ...          0             0
4                          0                      0  ...          0             0
...                      ...                    ...  ...        ...           ...
59510                      0                      0  ...          0             0
59511                      0                      0  ...          0             0
59512                      0                      0  ...          0             0
59513                      0                      0  ...          0             0
59514                      0                      0  ...          0             0

[59515 rows x 3

# Train net:

In [0]:
model = keras.Sequential()
model.add(Dense(150, input_dim=3*x_train_oh.shape[1], activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train_oh.values, y, epochs=100, batch_size=32, validation_split=.2)